To get around Colab's Markdown display issues and potential issues with the `plotly` library, the questions in this problem set are rendered in code blocks. Simply re-run the associated code block to re-render the question, if needed.

As a benefit, the questions will come with occasional hints about your programming environment.

Make sure the file `PS7.py` is in the same directory as this notebook. This file contains the question definitions and helper functions for this dataset. Do not modify this file.

If you do use Google Colab, make the notebook editable for the TAs, and provide the share link to the notebook below. Please share the notebook you worked on, and not a copy of it.

**Make sure the 'notify' checkbox is <i>not</i> checked when sharing the notebook.**

--------


link to Colab: https://colab.research.google.com/github/emilklo/amlberkeley/blob/main/PS7/INFO251_S25_PS7.ipynb#scrollTo=VtHeSO2bqyuP

(leave blank if not using Colab)

In [ ]:
# download necessary packages
%pip install sentence-transformers plotly datasets transformers "transformers[torch]"

In [ ]:
import matplotlib.pyplot as plt
import tqdm
import numpy as np
import seaborn as sns
sns.set_style()

import torch.nn as nn
import torch.nn.functional as F

import plotly.express as px

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
!wget https://raw.githubusercontent.com/emilklo/amlberkeley/main/PS7/PS7.py
!pip install datasets


import PS7

In [ ]:
PS7.intro()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
PS7.q0()

<span style="color:#FDB515"><b>Question 0 - your answers here</b></span>



In [ ]:
PS7.part1()

In [ ]:
PS7.q1()

In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsgroups = fetch_20newsgroups(
    subset='all', remove=('headers', 'footers', 'quotes')
)

# your code here
print(f"Number of documents: {len(newsgroups.data)}")
print(f"Categories: {newsgroups.target_names}")
print("Example document:")
print(newsgroups.data[0])
print(f"Label: {newsgroups.target_names[newsgroups.target[0]]}")


<span style="color:#FDB515"><b>Question 1 - your answers here</b></span>

a) The data is a collection of about 19,000 newsgroup posts from 20 online discussion topics. Each data point is a newsgroup post text document. Every document is labeled with one of 20 newsgroup categories.

b) For each text post, the prediction target is the category it belongs to.
This is represented by the target field in the dataset as an integer which maps to the category in target_names. There are 20 classes to predict.

c) No, newsgroups["data"] cannot be passed directly to a LogisticRegression method. newsgroups["data"] is a list of raw text strings, and LogisticRegression expects numerical input.



In [ ]:
PS7.q2()

In [ ]:
from sklearn.datasets import fetch_20newsgroups_vectorized

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

subset_size = 2000

# your code here
vectorized = fetch_20newsgroups_vectorized(subset='all')
X_tfidf = vectorized.data[:subset_size].toarray()
print(X_tfidf.shape)

raw_data = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
texts = raw_data.data[:subset_size]

import torch

if torch.cuda.is_available():
	device = 'cuda'
elif torch.backends.mps.is_available():
	device = 'mps'
else:
	device = 'cpu'

print(device)
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

truncation_limit = 256
num_truncated = 0
for text in texts:
    num_tokens = len(tokenizer.encode(text, truncation=True, add_special_tokens=True))
    if num_tokens >= truncation_limit:
        num_truncated += 1
print(f"Estimated number of truncated texts: {num_truncated} out of {subset_size}")


!pip install -q sentence-splitter
from sentence_splitter import SentenceSplitter
splitter = SentenceSplitter(language='en')

from tqdm import tqdm
import numpy as np

document_embeddings = []

for doc in tqdm(texts[:subset_size]):
    sentences = splitter.split(doc)
    if len(sentences) == 0:
        document_embeddings.append(np.zeros(384))
        continue
    sentence_embeddings = model.encode(sentences, batch_size=16, device=device)
    doc_embedding = np.mean(sentence_embeddings, axis=0)
    document_embeddings.append(doc_embedding)

X_embed = np.vstack(document_embeddings)
print("X_embed shape:", X_embed.shape)


<span style="color:#FDB515"><b>Question 2 - your answers here</b></span>

a) 484 data points out of the first 2000 run into the models truncation limit

b) (2000, 130107) and (2000, 384)

In [ ]:
PS7.q3()

In [ ]:
from sklearn.decomposition import PCA

# your code here
pca_tfidf = PCA(n_components=100, svd_solver='randomized', random_state=42)
var_tfidf = pca_tfidf.fit(X_tfidf).explained_variance_ratio_

plt.figure()
plt.plot(range(1, 101), var_tfidf)
plt.title('TF-IDF')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.show()

pca_embed = PCA(n_components=100, svd_solver='randomized', random_state=42)
var_embed = pca_embed.fit(X_embed).explained_variance_ratio_

plt.figure()
plt.plot(range(1, 101), var_embed)
plt.title('Embeddings')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.show()

# ——— Variation lost if only first 3 PCs are kept ———
loss_tfidf = 1 - var_tfidf[:3].sum()
loss_embed = 1 - var_embed[:3].sum()

print(f"TF-IDF: Variation lost with 3 PCs: {loss_tfidf:.2%}")
print(f"Embeddings:Variation lost with 3 PCs: {loss_embed:.2%}")

<span style="color:#FDB515"><b>Question 3 - your answers here</b></span>

a) For TF-IDF the curve starts above 7% for the first PC, then drops to around 2%, and continues a very gradual decreasing tail towards near 0 by PC 100. This tail shows that the tf-idf space is very high-dimensional and variance is spread over thousands of features, so each component beyond the first few captures only a tiny sliver of variance.

For the embeddings variant the first PC captures roughly 10%, the second about 3.2 %, the third 2.0 %, and then it falls off more steeply before settling into a lower tail. This indicates the 384-dimensional embedding space is much more compressible: a handful of PCs explain a sizeable chunk of the variance, unlike the tf-idf case.

b) TF-IDF
You lose about 88.3 % of the total variance, retaining only ∼11.7 % in the first three components.

Embeddings
You lose about 79.8 % of the variance, retaining ∼20.2 % in the first three components.

In [ ]:
PS7.q4()

In [ ]:
import plotly.express as px

# your code here
labels = raw_data.target[:subset_size]
names = raw_data.target_names

# ——— PCA to 3 dims ———
pca3_tfidf = PCA(n_components=3, random_state=42)
X_tfidf_3 = pca3_tfidf.fit_transform(X_tfidf)

pca3_embed = PCA(n_components=3, random_state=42)
X_embed_3 = pca3_embed.fit_transform(X_embed)

# ——— Verify explained‐variance sums ———
sum3_from100_tfidf = var_tfidf[:3].sum()                  # from your 100‐PC run
sum3_from3_tfidf  = pca3_tfidf.explained_variance_ratio_.sum()
print(f"TF-IDF: 100‐PC sum3 = {sum3_from100_tfidf:.4f}, 3‐PC fit sum = {sum3_from3_tfidf:.4f}")

sum3_from100_embed = var_embed[:3].sum()
sum3_from3_embed  = pca3_embed.explained_variance_ratio_.sum()
print(f"Embed: 100‐PC sum3 = {sum3_from100_embed:.4f}, 3‐PC fit sum = {sum3_from3_embed:.4f}")

# ——— 3D Scatter for TF-IDF ———
fig_tfidf = px.scatter_3d(
    x=X_tfidf_3[:,0], y=X_tfidf_3[:,1], z=X_tfidf_3[:,2],
    color=[names[i] for i in labels],
    title="3D PCA of TF-IDF Features",
    labels={'x':'PC1','y':'PC2','z':'PC3'}
)
fig_tfidf.show()

# ——— 3D Scatter for Embeddings ———
fig_embed = px.scatter_3d(
    x=X_embed_3[:,0], y=X_embed_3[:,1], z=X_embed_3[:,2],
    color=[names[i] for i in labels],
    title="3D PCA of Sentence Embeddings",
    labels={'x':'PC1','y':'PC2','z':'PC3'}
)
fig_embed.show()

<span style="color:#FDB515"><b>Question 4 - your answers here</b></span>

a) Yes. When I sum the first three explained‐variance ratios from the 100-component PCA and compare it to the variance from a direct 3-component PCA, I get the exact same numbers. This shows that PCA always allocates the same total variance to the top k components whether I ask for k directly or slice them off a larger decomposition.

b) Yes. The first three eigenvectors of the covariance matrix are the same for 3 or 100.

c) For the tf-idf plot, the data points almost looks like they are scattered randomly. Light green for example, can be seen top left and right, bottom left and right, and center. Clusters bleed into each other because raw word count patterns capture surface overlap. “drive” appears in both comp.sys.ibm.pc.hardware and rec.motorcycles.

For the embeddings plot, the clusters are noticeably tighter and more separated, reflecting that the MiniLM model has learned deeper semantic distinctions. It looks like some groups have clustered together, indicating that some topics have similarities with each other.

In [ ]:
PS7.part2()

In [ ]:
PS7.q5()

In [ ]:
from sklearn.metrics import pairwise_distances_argmin

def k_means_clustering(X, k, random_state=None):
    # your code here
    rng = np.random.default_rng(random_state)
    # choosing centroids
    init_idx = rng.choice(X.shape[0], size=k, replace=False)
    centroids = X[init_idx]

    for i in range(1, 101):
        labels = pairwise_distances_argmin(X, centroids)

        new_centroids = np.vstack([
            X[labels == j].mean(axis=0) if np.any(labels == j) else centroids[j]
            for j in range(k)
        ])

        # convergance check
        shift = np.linalg.norm(new_centroids - centroids, axis=1).max()
        centroids = new_centroids
        if shift < 1e-4:
            break

    return centroids, labels, i

k = 20
centroids, labels, n_iters = k_means_clustering(X_embed_3, k)

print(f"Converged in {n_iters} iterations")
print(f"Centroids shape: {centroids.shape}")
print(f"Labels shape: {labels.shape}")

In [ ]:
PS7.q6()

In [ ]:
from sklearn.datasets import make_blobs

X, y_true = make_blobs(n_samples=300, centers=4, cluster_std=0.60, random_state=0)
# visualize the synthetic dataset to choose the number of clusters
# plt.scatter(X[:, 0], X[:, 1], s=50)

# your code here
# Run custom k-means
k = 4
centroids, labels, n_iters = k_means_clustering(X, k)

# Plot the results
plt.figure(figsize=(8, 6))
scatter = plt.scatter(X[:, 0], X[:, 1], c=labels, s=50, cmap='tab10', edgecolor='k', alpha=0.7)
plt.scatter(centroids[:, 0], centroids[:, 1], c='black', marker='x', s=200, linewidths=3, label='Centroids')
plt.title(f'K-Means Clustering on Synthetic Data (k={k})')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.show()

In [ ]:
PS7.q7()

In [ ]:
# your code here
def davies_bouldin_index(X, labels, centroids):
    k = centroids.shape[0]
    # 1) Compute scatter S_i for each cluster = average distance to its centroid
    S = np.zeros(k)
    for i in range(k):
        members = X[labels == i]
        if len(members) > 0:
            dists = np.linalg.norm(members - centroids[i], axis=1)
            S[i] = dists.mean()
        else:
            S[i] = 0.0

    # 2) Compute R_ij = (S_i + S_j) / d(c_i, c_j)
    #    then R_i = max_{j != i} R_ij
    R = np.zeros((k, k))
    D = np.linalg.norm(centroids[:, None] - centroids[None, :], axis=2)  # pairwise centroid–centroid
    for i in range(k):
        for j in range(k):
            if i != j and D[i, j] > 0:
                R[i, j] = (S[i] + S[j]) / D[i, j]
    R_i = R.max(axis=1)

    # 3) DB index = average of R_i
    return R_i.mean()

# ——— Sweep k from 2 to 15 ———
db_scores = []
ks = list(range(2, 16))
for k in ks:
    centroids, labels, _ = k_means_clustering(X_embed_3, k)    # your k_means on 3-D data
    db = davies_bouldin_index(X_embed_3, labels, centroids)
    db_scores.append(db)

# ——— Plot DB index vs k ———
plt.figure(figsize=(6,4))
plt.plot(ks, db_scores, marker='o')
plt.xticks(ks)
plt.xlabel("Number of clusters k")
plt.ylabel("Davies–Bouldin index (lower is better)")
plt.title("DB Index vs k on 3-D PCA Embeddings")
plt.grid(True)
plt.show()


<span style="color:#FDB515"><b>Question 7 - your answers here</b></span>

c) According to the index, the sweet spot is 8 clusters, which is fewer than the 20 original newsgroup labels. This tells us that while there are twenty topics, the data really condenses into a handful of broader themes. It aligns perfectly with what q4c told us: documents naturally group into larger topics like computer forums, science, and recreation rather than twenty completely separate islands. The eight clusters capture the major semantic divides observed in the 3D PCA plot for embeddings.

In [ ]:
PS7.part3()

In [ ]:
PS7.q8()

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score
from PS7 import newsgroups_tokenized_datasets

# tokenize the training and validation datasets and handle padding tokens (done for you)
base_model = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(base_model)
trn_dataset, val_dataset = newsgroups_tokenized_datasets(tokenizer)

In [ ]:
# instantiate the model

# hint: pass in pad_token_id=tokenizer.pad_token_id
model = AutoModelForSequenceClassification.from_pretrained(
    base_model,
    num_labels=20,
    pad_token_id=tokenizer.pad_token_id
)

# hyperparameters
args = TrainingArguments(
    output_dir="./ft-distilgpt2",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_strategy="no",
    logging_steps=100,
    report_to="none",
    load_best_model_at_end=True
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, preds)}

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=trn_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
metrics = trainer.evaluate()
print(f"Validation accuracy: {metrics['eval_accuracy']:.2%}")

In [ ]:
train_metrics = trainer.evaluate(eval_dataset=trn_dataset)
print(f"Train accuracy: {train_metrics['eval_accuracy']:.2%}")

<span style="color:#FDB515"><b>Question 8 - your answers here</b></span>

hyperparameters:

Epochs: 3

Batch sizes: 16

Learning rate: 2 * 10^-5

Weight decay: 0.01

train accuracy: 76.37%

test accuracy: 67.26%

In [ ]:
PS7.q9()

In [ ]:
# your code here
from sklearn.linear_model import LogisticRegressionCV

tfidf_train = fetch_20newsgroups_vectorized(subset='train')
X_tfidf_train = tfidf_train.data.toarray()
y_train = tfidf_train.target

tfidf_test = fetch_20newsgroups_vectorized(subset='test')
X_tfidf_test = tfidf_test.data.toarray()
y_test = tfidf_test.target

lr_tfidf = LogisticRegressionCV(
    Cs=[0.01, 0.1, 1, 10],
    cv=5,
    penalty='l2',
    solver='saga',
    max_iter=1000,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42
)
lr_tfidf.fit(X_tfidf_train, y_train)

train_acc_tfidf = lr_tfidf.score(X_tfidf_train, y_train)
test_acc_tfidf = lr_tfidf.score(X_tfidf_test,  y_test)
best_C_tfidf = lr_tfidf.C_[0]

model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')

raw_train = fetch_20newsgroups(subset='train', remove=('headers','footers','quotes'))
X_embed_train = model.encode(raw_train.data, batch_size=32, show_progress_bar=True)
y_train_embed = raw_train.target

raw_test = fetch_20newsgroups(subset='test', remove=('headers','footers','quotes'))
X_embed_test = model.encode(raw_test.data,  batch_size=32, show_progress_bar=True)
y_test_embed = raw_test.target

lr_embed = LogisticRegressionCV(
    Cs=[0.01, 0.1, 1, 10],
    cv=5,
    penalty='l2',
    solver='saga',
    max_iter=1000,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42
)
lr_embed.fit(X_embed_train, y_train_embed)

train_acc_embed = lr_embed.score(X_embed_train, y_train_embed)
test_acc_embed = lr_embed.score(X_embed_test,  y_test_embed)
best_C_embed = lr_embed.C_[0]

# ——— 5) Report results ———
print("TF-IDF Logistic Regression")
print(f" • Best C      = {best_C_tfidf}")
print(f" • Train acc   = {train_acc_tfidf:.2%}")
print(f" • Test  acc   = {test_acc_tfidf:.2%}\n")

print("MiniLM Embeddings Logistic Regression")
print(f" • Best C      = {best_C_embed}")
print(f" • Train acc   = {train_acc_embed:.2%}")
print(f" • Test  acc   = {test_acc_embed:.2%}")

<span style="color:#FDB515"><b>Question 9 - your answers here</b></span>

tf-idf hyperparameters:

tf-idf train accuracy:

tf-idf test accuracy:

miniLM hyperparameters:

miniLM train accuracy:

miniLM test accuracy:

discussion:

In [ ]:
PS7.ec1()

In [ ]:
# your code here

<span style="color:#FDB515"><b>Extra credit 1 - your answers here</b></span>

a)

b)


In [ ]:
PS7.ec2()

In [ ]:

class SelfAttention(nn.Module):
    def __init__(self, d_embedding):
        """
        d_embedding: int, the dimension of the input embedding
        """
        super().__init__()
        # your code here
        ...


    def forward(self, x):
        # your code here
        ...

def attention_classifier(num_classes):
    # create an embedding layer and freeze it so that it won't be updated
    embed = nn.Embedding.from_pretrained(..., freeze=True)
    d_embedding = embed.embedding_dim

    # your implemented attention layer
    attn = SelfAttention(d_embedding=d_embedding)

    # classifier head
    return nn.Sequential(
        embed,                      # tokens -> word vectors
        attn,                       # self‑attention
        nn.AdaptiveAvgPool1d(1),    # mean-pool over the sequence of words
        nn.Flatten(),
        nn.Linear(d_embedding, num_classes)  # classifier
    )


In [ ]:
PS7.ec3()

In [ ]:
# your code here

<span style="color:#FDB515"><b>Extra credit 3 - your answers here</b></span>

